# Workspace for Project Euler problems

## Problem 9

A Pythagorean triplet is a set of three natural numbers, $a < b < c$, for which,

$$a^2 + b^2 = c^2$$

For example, $3^2 + 4^2 = 9 + 16 = 25 = 5^2$.

There exists exactly one Pythagorean triplet for which $a + b + c = 1000$.
Find the product $abc$.

In [9]:
import primetools as pt
import numbers, math
import tools.misc as misc

In [13]:
def squares(mx):
    sq=[i*2 for i in range(1, math.floor(math.sqrt(max)))]
    return sq

In [11]:
squares(300)

TypeError: must be real number, not builtin_function_or_method